# UKW Marine Radio Chatter - Bridge 2 Bridge Communication
This notebook uses pretrained models to transcribe the audio files from the UKW Marine Radio Chatter - Bridge 2 Bridge Communication dataset. <br>
The dataset contains audio files and their corresponding transcriptions. Further we classify the speakers contained in the audio files.

In [1]:
import os
import IPython
import torchaudio
import torch
from pydub import AudioSegment
from pytorch_lightning.utilities.types import STEP_OUTPUT
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from src.utils import txt_to_dataframe
import numpy as np

## Configuration - Data Directories

In [2]:
class Config:
    DATA_DIR = '../data/'
    AUDIO_DIR = DATA_DIR + 'audio/'
    TEXT_DIR = DATA_DIR + 'text/'
    DATASET_DIR = 'dataset/'
    
    KAGGLE_DATA_TAG = 'linogova/marine-radio-chatter-bridge-2-bridge-communication/1'
    KAGGLE_DATA_DIR = 'Marine_audio/'

config = Config()

In [3]:
import pandas as pd

audio_files = [d + "/" + f for d in os.listdir(config.AUDIO_DIR) for f in os.listdir(config.AUDIO_DIR + "/" + d) if f]
text_files = [f.replace(".mp3", ".txt") for f in audio_files]

audio_files = [config.AUDIO_DIR + f for f in audio_files]
text_files = [config.TEXT_DIR + f for f in text_files]

audio_text_files = zip(audio_files, text_files)
audio_text_files = [(a, t) for a, t in audio_text_files if os.path.isfile(a) and os.path.isfile(t)]

df = pd.DataFrame(audio_text_files, columns=['audio_file', 'text_file'])
df.head()

,audio_file,text_file
0,../data/audio/26694/26694-20231228-0414.mp3,../data/text/26694/26694-20231228-0414.txt
1,../data/audio/26694/26694-20231225-0133.mp3,../data/text/26694/26694-20231225-0133.txt
2,../data/audio/26694/26694-20231214-0023.mp3,../data/text/26694/26694-20231214-0023.txt
3,../data/audio/26694/26694-20231216-1710.mp3,../data/text/26694/26694-20231216-1710.txt
4,../data/audio/26694/26694-20231221-1759.mp3,../data/text/26694/26694-20231221-1759.txt


In [4]:
def convert_mp3_to_wav(in_fpath, out_fpath):
    audio = AudioSegment.from_mp3(in_fpath)
    audio = audio.set_frame_rate(16000)
    audio.export(out_fpath, format="wav")
    return out_fpath


# for index, row in df.head(1000).iterrows():
#     wav_file_path = config.DATASET_AUDIO + row['audio_file'].split("/")[-1].replace(".mp3", ".wav")
#     csv_file_path = config.DATASET_TEXT + row['text_file'].split("/")[-1].replace(".txt", ".csv")
#     convert_mp3_to_wav(row['audio_file'], wav_file_path)
# 
#     sample_df = txt_to_dataframe(row['text_file'])
#     sample_df.to_csv(csv_file_path, index=False)

In [5]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torchaudio
from joblib import Parallel, delayed
from torch.utils.data import Dataset
from transformers import WhisperProcessor
from src.utils import bcolors

c = bcolors()

def batch_data(data, max_duration=30):
    batches = []
    current_batch = []
    curr_start = 0

    for entry in data:
        if entry['end_time'] - curr_start + 0.2  > max_duration:
            if len(current_batch) == 0:
                curr_start = entry['start_time'] - 0.2
            else:
                batches.append(current_batch)
                current_batch = []
                curr_start = entry['start_time'] - 0.2

        current_batch.append(entry)

    if current_batch:
        batches.append(current_batch)

    return batches

def inner_merge_batches(data):
    return [{
        'text': ' '.join([entry['transcript'] for entry in batch]),
        'start': batch[0]['start_time'],
        'end': batch[-1]['end_time']
    } for i, batch in enumerate(data)]

class UKWFunkSprache(Dataset):
    def __init__(self, 
                 file_ids, 
                 root_dir, 
                 proc=None, 
                 n_jobs=-1):
        self.feed_ids = file_ids
        self.root_dir = root_dir
        self.processor = proc

        print(f"\n{c.OKGREEN}Preloading Samples...{c.ENDC}")
        print(f"\n{c.OKCYAN}Audio Files:         {len(self.feed_ids)}{c.ENDC}")
        print(f"{c.OKCYAN}Jobs:                {n_jobs} {c.ENDC}\n")

        start_time = time.time()
        result = Parallel(n_jobs=n_jobs)(
            delayed(self.process_file)(idx) for idx in range(len(self.feed_ids))
        )
        result = [item for sublist in result for item in sublist]
        print(f"\n{c.OKGREEN}Preloading Complete!{c.ENDC}")

        self.audio_samples = [item['audio'] for item in result]
        self.transcriptions = [item['transcript'] for item in result]
        self.groups = [item['group'] for item in result]
        
        print(f"{c.OKCYAN}Number of Samples:   {len(self.audio_samples)} {c.ENDC}\n")

        end_time = time.time()
        t = end_time - start_time
        print(f"\n{c.OKBLUE}Time taken:      {int((t - (t % 60)) / 60)} min {t % 60} sec {c.ENDC}")

    def process_file(self, idx):
        feed_id = self.feed_ids[idx]
        audio_fpath = os.path.join(self.root_dir, f"audio/{feed_id}.wav")
        text_fpath = os.path.join(self.root_dir, f"text/{feed_id}.csv")

        waveform, sample_rate = torchaudio.load(audio_fpath)
        transcripts_df = pd.read_csv(text_fpath)
        
        if len(transcripts_df) < 5:
            return []
        
        batches = batch_data(transcripts_df.to_dict('records'))
        metadata = inner_merge_batches(batches)

        sample_group = str(feed_id)
        samples = []
        for i in range(len(metadata)):
            start_time = metadata[i]['start'] - 0.2
            end_time = metadata[i]['end'] + 0.2
            transcript = metadata[i]['text']
            
            if end_time - start_time < 10:
                continue

            start_sample = int(start_time * sample_rate)
            end_sample = int(end_time * sample_rate)

            sample = waveform[:, start_sample:end_sample].squeeze().numpy()
            
            
            if self.processor:
                sample = self.processor.feature_extractor(sample, sampling_rate=sample_rate, return_tensors="pt").input_features.squeeze(0)
                transcript = self.processor.tokenizer(transcript, return_tensors="pt").input_ids.squeeze(0)

            samples.append({
                'group': sample_group,
                'audio': sample,
                'transcript': transcript
            })
        return samples

    def __len__(self):
        return len(self.audio_samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        audio = self.audio_samples[idx]
        transcript = self.transcriptions[idx]

        return {
            "input_features": audio,
            "labels": transcript
        }


# Model

In [6]:
import torchmetrics
from transformers import get_linear_schedule_with_warmup
import pytorch_lightning as pl

class WhisperLightningModule(pl.LightningModule):
    def __init__(self, model_name: str, processor, learning_rate: float, weight_decay: float, warmup_steps: int):
        super().__init__()
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name)
        self.processor = processor
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.warmup_steps = warmup_steps
        
        self.wer = torchmetrics.text.wer.WordErrorRate()

    def forward(self, input_features, labels):
        return self.model(input_features=input_features, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_features"], batch["labels"])
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_features"], batch["labels"])
        loss = outputs.loss
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        
    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_features"], batch["labels"])
        loss = outputs.loss

        # Compute WER for samples in the batch
        pred_tok_ids = torch.argmax(outputs.logits, dim=-1)
        dec_preds = self.processor.batch_decode(pred_tok_ids)
        dec_labels = self.processor.batch_decode(batch["labels"])
        wer_score = wer(dec_preds, dec_labels)
        
        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("test_wer", wer_score, prog_bar=True, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.warmup_steps,
            num_training_steps=self.trainer.estimated_stepping_batches
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

In [7]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class SpeechDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, processor, batch_size: int, num_workers: int = 8):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.processor = processor
        self.batch_size = batch_size
        self.num_workers = num_workers

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, collate_fn=self.collate_fn, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, collate_fn=self.collate_fn, num_workers=self.num_workers)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, collate_fn=self.collate_fn, num_workers=self.num_workers)

    def collate_fn(self, features):
        input_features = [feature["input_features"] for feature in features]
        batch = self.processor.feature_extractor.pad(
            [{"input_features": input_feature} for input_feature in input_features],
            return_tensors="pt"
        )

        labels = [feature["labels"] for feature in features]
        labels_batch = self.processor.tokenizer.pad(
            [{"input_ids": label} for label in labels],
            return_tensors="pt"
        )
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.pad_token_id).all():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch


In [8]:
from tabulate import tabulate

def print_parameter_table(model):
    # Collect parameter information
    parameter_info = []
    total_params = 0
    # only append parameters that require gradients, if they are frozen, summarize them as well and put them at the end
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_params = param.numel()
            total_params += num_params
            parameter_info.append([name, param.requires_grad, num_params])

    # Print table
    headers = ["Parameter Name", "Requires Grad", "Num Params"]
    print(tabulate(parameter_info, headers=headers, tablefmt="grid"))

    # Print total number of parameters
    print(f"Total Parameters: {total_params}")

# Finetuning

In [9]:
from transformers import WhisperProcessor

# Initialize the processor
processor = WhisperProcessor.from_pretrained(model_name, language='en', task="transcribe")

# Create the Datasets
feed_ids = [f.replace(".wav", "") for f in os.listdir(config.DATASET_DIR + "audio")]

ds_train = UKWFunkSprache(feed_ids[:800], config.DATASET_DIR, proc=processor)
ds_val = UKWFunkSprache(feed_ids[800:1000], config.DATASET_DIR, proc=processor)

NameError: name 'model_name' is not defined

In [ ]:
import pytorch_lightning as pl


# Initialize processor and model name
model_name = "openai/whisper-tiny"


# Define training arguments
learning_rate = 1e-4
weight_decay = 0.01
warmup_steps = 100
batch_size = 8
num_epochs = 5

In [ ]:
from transformers import WhisperForConditionalGeneration

# Initialize DataModule
data_module = SpeechDataModule(ds_train, ds_val, processor, batch_size)

# Initialize the model
model_train = WhisperLightningModule(model_name, processor, learning_rate, weight_decay, warmup_steps)

model_train.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model_train.model.generation_config.language = "en"
model_train.model.generation_config.task = "transcribe"

# Freeze layers in the encoder
for param in model_train.model.model.encoder.parameters():
    param.requires_grad = False

# Freeze layers in the decoder
for param in model_train.model.model.decoder.parameters():
    param.requires_grad = False

# Unfreeze the final linear layer for fine-tuning
model_train.model.proj_out.weight.requires_grad = True

# Set requires_grad to True to fine-tune embeddings
for name, param in model_train.model.named_parameters():
    if "embed" in name:
        param.requires_grad = True  

print_parameter_table(model_train.model)


In [ ]:
from pytorch_lightning.loggers import WandbLogger
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint

# logger = WandbLogger(
#     project="eurosat_" + model_name,
#     name=model_name,
#     log_model=False,
# )

# Define callbacks (like ModelCheckpoint)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="outputs/whisper-tiny/" + datetime.now().strftime("%H-%M"),
    filename="{epoch:02d}-{train_loss_epoch:.2f}-{val_loss:.2f}",
    save_top_k=2,
    mode="min"
)

# Initialize the Trainer
trainer = pl.Trainer(
    max_epochs=num_epochs,
    callbacks=[checkpoint_callback],
    logger=False,
    accelerator="auto",
    log_every_n_steps=5
)

# Train the model
trainer.fit(model_train, data_module)

In [ ]:
trainer.test(model_train, data_module, ckpt_path=checkpoint_callback.best_model_path)

In [ ]:

model_test = WhisperLightningModule.load_from_checkpoint(
    checkpoint_callback.best_model_path, 
    model_name=model_name, 
    processor=processor, 
    learning_rate=learning_rate, 
    weight_decay=weight_decay, 
    warmup_steps=warmup_steps
)

model_test.model.cuda()
i = 0
for batch in data_module.val_dataloader():
    pred = model_test(batch["input_features"].to("cuda"), batch["labels"].to("cuda"))
    pred_token_ids = torch.argmax(pred.logits, dim=-1)
    decoded_labels = processor.batch_decode(batch["labels"], skip_special_tokens=True)
    decoded_preds = processor.batch_decode(pred_token_ids, skip_special_tokens=True)
    
    wer = torchmetrics.text.wer.WordErrorRate()
    wer(decoded_preds, decoded_labels)
    print(wer.compute())
    
    print("Predicted:\n", decoded_preds[0])
    print("True:\n", decoded_labels[0])
    print()
    i += 1
    if i == 10:
        break
